# Inspect identified impact hotspots in African countries

* **Products used:** [DE Africa Coastlines]()



### Load packages
Import Python packages that are used for the analysis.

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import matplotlib as mpl

In [2]:
import glob

hotspots_files = glob.glob('../coastchange_hotspots_obselete/*_*_hotspots.geojson')
hotspots_files = [hotspots_file for hotspots_file in hotspots_files if not hotspots_file.startswith('hotspots')]

In [3]:
hotspots=[]

for hotspots_file in hotspots_files:
    hotspots.append(gpd.read_file(hotspots_file))

In [4]:
hotspots_all = pd.concat(hotspots)

In [5]:
hotspots_all= hotspots_all.to_crs("EPSG:6933")

In [6]:
hotspots_all['total_pop']=hotspots_all.area*hotspots_all["pop"]/30/30

In [7]:
hotspots_all[['pop','total_pop']].sum()

pop           24766.267122
total_pop    227632.525617
dtype: float64

In [8]:
hotspots_pop1000 = hotspots_all[hotspots_all['total_pop']>1000] #filter for high confidence areas
hotspots_pop100 = hotspots_all[hotspots_all['total_pop']>100]

In [9]:
gadm_africa  = gpd.read_file("../data/gadm_africa.gpkg")

In [10]:
gadm_level0 = gadm_africa.dissolve(by=f"GID_0", as_index=False)
gadm_level0 = gadm_level0.to_crs(hotspots_pop1000.crs)
gadm_level1 = gadm_africa.dissolve(by=f"GID_1", as_index=False)
gadm_level1 = gadm_level1.to_crs(hotspots_pop1000.crs)

In [11]:
gadm_level1#[["NAME_0","geometry"]]

,GID_1,geometry,UID,GID_0,NAME_0,VARNAME_0,NAME_1,VARNAME_1,NL_NAME_1,ISO_1,...,GOVERNEDBY,SOVEREIGN,DISPUTEDBY,REGION,VARREGION,COUNTRY,CONTINENT,SUBCONT,layer,path
0,,"MULTIPOLYGON (((-515142.952 4319024.815, -5151...",103975,GIB,Gibraltar,,,,,,...,,United Kingdom,Spain,,,Gibraltar,Africa,,gadm_africa,C:\Users\Fang Yuan\OneDrive - FrontierSI\Docum...
1,AGO.10_1,"POLYGON ((1337578.897 -2038881.079, 1335697.89...",2574,AGO,Angola,,Huíla,,,,...,,Angola,,,,Angola,Africa,,gadm_africa,C:\Users\Fang Yuan\OneDrive - FrontierSI\Docum...
2,AGO.11_1,"MULTIPOLYGON (((1280067.856 -1166669.344, 1278...",2609,AGO,Angola,,Luanda,Loanda,,AO-LUA,...,,Angola,,,,Angola,Africa,,gadm_africa,C:\Users\Fang Yuan\OneDrive - FrontierSI\Docum...
3,AGO.12_1,"POLYGON ((1874305.731 -1271982.925, 1871843.73...",2636,AGO,Angola,,Lunda Norte,,,,...,,Angola,,,,Angola,Africa,,gadm_africa,C:\Users\Fang Yuan\OneDrive - FrontierSI\Docum...
4,AGO.13_1,"POLYGON ((1928191.69 -1428331.723, 1927307.78 ...",2662,AGO,Angola,,Lunda Sul,,,,...,,Angola,,,,Angola,Africa,,gadm_africa,C:\Users\Fang Yuan\OneDrive - FrontierSI\Docum...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
878,ZWE.5_1,"POLYGON ((3020295.985 -2404184.844, 3020302.64...",355148,ZWE,Zimbabwe,,Mashonaland East,,,ZW-ME,...,,Zimbabwe,,,,Zimbabwe,Africa,,gadm_africa,C:\Users\Fang Yuan\OneDrive - FrontierSI\Docum...
879,ZWE.6_1,"POLYGON ((2853604.495 -2303221.475, 2853221.16...",355377,ZWE,Zimbabwe,,Mashonaland West,,,ZW-MW,...,,Zimbabwe,,,,Zimbabwe,Africa,,gadm_africa,C:\Users\Fang Yuan\OneDrive - FrontierSI\Docum...
880,ZWE.7_1,"POLYGON ((2944204.594 -2705624.596, 2944193.69...",355607,ZWE,Zimbabwe,,Masvingo,,,ZW-MV,...,,Zimbabwe,,,,Zimbabwe,Africa,,gadm_africa,C:\Users\Fang Yuan\OneDrive - FrontierSI\Docum...
881,ZWE.8_1,"POLYGON ((2508219.371 -2384702.939, 2508225.83...",355850,ZWE,Zimbabwe,,Matabeleland North,,,ZW-MN,...,,Zimbabwe,,,,Zimbabwe,Africa,,gadm_africa,C:\Users\Fang Yuan\OneDrive - FrontierSI\Docum...


In [12]:
hotspots_pop1000=hotspots_pop1000.sjoin_nearest(gadm_level0[["GID_0", "NAME_0","geometry"]], how="left", max_distance=100)
hotspots_pop1000 = hotspots_pop1000.drop(columns=["index_right"])
hotspots_pop1000=hotspots_pop1000.sjoin_nearest(gadm_level1[["GID_1", "NAME_1","geometry"]], how="left", max_distance=100)
hotspots_pop1000 = hotspots_pop1000.drop(columns=["index_right"])

In [13]:
hotspots_pop100=hotspots_pop100.sjoin_nearest(gadm_level0[["GID_0", "NAME_0","geometry"]], how="left", max_distance=100)
hotspots_pop100 = hotspots_pop100.drop(columns=["index_right"])
hotspots_pop100=hotspots_pop100.sjoin_nearest(gadm_level1[["GID_1", "NAME_1","geometry"]], how="left", max_distance=100)
hotspots_pop100 = hotspots_pop100.drop(columns=["index_right"])

In [14]:
hotspots_all=hotspots_all.sjoin_nearest(gadm_level0[["GID_0", "NAME_0","geometry"]], how="left", max_distance=100)
hotspots_all = hotspots_all.drop(columns=["index_right"])
hotspots_all = hotspots_all.sjoin_nearest(gadm_level1[["GID_1", "NAME_1","geometry"]], how="left", max_distance=100)
hotspots_all = hotspots_all.drop(columns=["index_right"])

In [15]:
grouped_country = hotspots_pop100.groupby('GID_0')['total_pop'].sum().reset_index()
grouped_country = grouped_country.rename(columns={'total_pop':'total_pop_sum'})
grouped_country['count']= hotspots_pop100.groupby('GID_0')['total_pop'].count().values
grouped_country['count_1']= hotspots_pop100.groupby('GID_0')['NAME_1'].nunique().values
grouped_country['NAME_0'] = hotspots_pop100.groupby('GID_0')['NAME_0'].first().values
top_countries = grouped_country.sort_values('total_pop_sum',ascending=False)[:5]['NAME_0']
print(top_countries.values)

['Nigeria' 'Benin' 'Gabon' 'Mozambique' 'Senegal']


In [16]:
grouped_country = hotspots_all.groupby('GID_0')['total_pop'].sum().reset_index()
grouped_country = grouped_country.rename(columns={'total_pop':'total_pop_sum'})
grouped_country['count']= hotspots_all.groupby('GID_0')['total_pop'].count().values
grouped_country['count_1']= hotspots_all.groupby('GID_0')['NAME_1'].nunique().values
grouped_country['NAME_0'] = hotspots_all.groupby('GID_0')['NAME_0'].first().values
top_countries = grouped_country.sort_values('total_pop_sum',ascending=False)[:5]['NAME_0']
print(top_countries.values)

['Nigeria' 'Benin' 'Mozambique' 'Gabon' 'Senegal']


In [17]:
grouped_pop1000 = hotspots_pop1000.groupby("GID_1")['total_pop'].sum().reset_index()
grouped_pop1000 = grouped_pop1000.rename(columns={'total_pop':'total_pop_sum'})
grouped_pop1000['count']= hotspots_pop1000.groupby("GID_1")['total_pop'].count().values
grouped_pop1000['NAME_0'] = hotspots_pop1000.groupby("GID_1")['NAME_0'].first().values
grouped_pop1000['NAME_1'] = hotspots_pop1000.groupby("GID_1")['NAME_1'].first().values
grouped_pop1000.sort_values('total_pop_sum',ascending=False)[:10]
gadm_level1_ppo1000 = pd.merge(gadm_level1[['GID_1', 'NAME_1', 'geometry', 'UID', 'GID_0', 'NAME_0']], grouped_pop1000[['GID_1', 'NAME_1', 'total_pop_sum','count']].sort_values('total_pop_sum',ascending=False)[:10], on='GID_1', how='right')
gadm_level1_ppo1000[gadm_level1_ppo1000['NAME_0'].isin(top_countries.values)]

,GID_1,NAME_1_x,geometry,UID,GID_0,NAME_0,NAME_1_y,total_pop_sum,count
0,NGA.29_1,Ondo,"MULTIPOLYGON (((481587.146 747668.981, 481587....",216760,NGA,Nigeria,Ondo,25144.668819,10
1,BEN.8_1,Littoral,"POLYGON ((233724.592 807684.583, 233376.103 80...",18156,BEN,Benin,Littoral,20776.083776,4
2,GAB.1_1,Estuaire,"MULTIPOLYGON (((916579.524 13661.05, 916713.44...",92270,GAB,Gabon,Estuaire,12895.594121,4
3,NGA.25_1,Lagos,"POLYGON ((262194.724 823787.854, 262235.625 82...",216682,NGA,Nigeria,Lagos,9192.135149,5
4,BEN.10_1,Ouémé,"POLYGON ((243855.654 809563.667, 243855.654 80...",18204,BEN,Benin,Ouémé,8452.661386,4
5,SEN.10_1,Saint-Louis,"POLYGON ((-1588223.527 1995300.864, -1588860.8...",290334,SEN,Senegal,Saint-Louis,7604.750757,3
6,MOZ.9_1,Sofala,"MULTIPOLYGON (((3379518.996 -2618388.182, 3378...",210939,MOZ,Mozambique,Sofala,4324.686304,3
9,MOZ.7_1,Nampula,"MULTIPOLYGON (((3775655.481 -2126800.805, 3775...",210833,MOZ,Mozambique,Nampula,2680.026041,2


In [18]:
grouped_pop100 = hotspots_pop100.groupby('GID_1')['total_pop'].sum().reset_index()
grouped_pop100 = grouped_pop100.rename(columns={'total_pop':'total_pop_sum'})
grouped_pop100['count']= hotspots_pop100.groupby('GID_1')['total_pop'].count().values
grouped_pop100['NAME_0'] = hotspots_pop100.groupby('GID_1')['NAME_0'].first().values
grouped_pop100['NAME_1'] = hotspots_pop100.groupby('GID_1')['NAME_1'].first().values
gadm_level1_ppo100 = pd.merge(gadm_level1[['GID_1', 'NAME_1', 'geometry', 'UID', 'GID_0', 'NAME_0']], grouped_pop100[['GID_1', 'NAME_1', 'total_pop_sum','count']].sort_values('total_pop_sum',ascending=False)[:10], on='GID_1', how='right')
gadm_level1_ppo100[gadm_level1_ppo100['NAME_0'].isin(top_countries.values)]

,GID_1,NAME_1_x,geometry,UID,GID_0,NAME_0,NAME_1_y,total_pop_sum,count
0,NGA.29_1,Ondo,"MULTIPOLYGON (((481587.146 747668.981, 481587....",216760,NGA,Nigeria,Ondo,38231.280586,43
1,BEN.8_1,Littoral,"POLYGON ((233724.592 807684.583, 233376.103 80...",18156,BEN,Benin,Littoral,21348.835848,7
2,GAB.1_1,Estuaire,"MULTIPOLYGON (((916579.524 13661.05, 916713.44...",92270,GAB,Gabon,Estuaire,14960.789868,9
3,NGA.25_1,Lagos,"POLYGON ((262194.724 823787.854, 262235.625 82...",216682,NGA,Nigeria,Lagos,13830.188236,21
4,NGA.33_1,Rivers,"MULTIPOLYGON (((666599.685 568576.775, 666626....",216858,NGA,Nigeria,Rivers,11134.961724,36
5,BEN.10_1,Ouémé,"POLYGON ((243855.654 809563.667, 243855.654 80...",18204,BEN,Benin,Ouémé,9994.660112,8
6,SEN.10_1,Saint-Louis,"POLYGON ((-1588223.527 1995300.864, -1588860.8...",290334,SEN,Senegal,Saint-Louis,8479.015525,6
7,MOZ.9_1,Sofala,"MULTIPOLYGON (((3379518.996 -2618388.182, 3378...",210939,MOZ,Mozambique,Sofala,7055.793116,15


In [19]:
grouped = hotspots_all.groupby('GID_1')['total_pop'].sum().reset_index()
grouped = grouped.rename(columns={'total_pop':'total_pop_sum'})
grouped['count']= hotspots_all.groupby('GID_1')['total_pop'].count().values
grouped['NAME_0'] = hotspots_all.groupby('GID_1')['NAME_0'].first().values
grouped['NAME_1'] = hotspots_all.groupby('GID_1')['NAME_1'].first().values
gadm_level1_top = pd.merge(gadm_level1[['GID_1', 'NAME_1', 'geometry', 'UID', 'GID_0', 'NAME_0']], grouped[['GID_1', 'NAME_1', 'total_pop_sum','count']].sort_values('total_pop_sum',ascending=False)[:10], on='GID_1', how='right')
gadm_level1_top[gadm_level1_top['NAME_0'].isin(top_countries.values)]

,GID_1,NAME_1_x,geometry,UID,GID_0,NAME_0,NAME_1_y,total_pop_sum,count
0,NGA.29_1,Ondo,"MULTIPOLYGON (((481587.146 747668.981, 481587....",216760,NGA,Nigeria,Ondo,41474.089289,163
1,BEN.8_1,Littoral,"POLYGON ((233724.592 807684.583, 233376.103 80...",18156,BEN,Benin,Littoral,21707.592483,19
2,NGA.25_1,Lagos,"POLYGON ((262194.724 823787.854, 262235.625 82...",216682,NGA,Nigeria,Lagos,16714.585484,234
3,GAB.1_1,Estuaire,"MULTIPOLYGON (((916579.524 13661.05, 916713.44...",92270,GAB,Gabon,Estuaire,15300.592224,19
4,NGA.33_1,Rivers,"MULTIPOLYGON (((666599.685 568576.775, 666626....",216858,NGA,Nigeria,Rivers,13619.932059,90
5,BEN.10_1,Ouémé,"POLYGON ((243855.654 809563.667, 243855.654 80...",18204,BEN,Benin,Ouémé,11089.792015,65
6,MOZ.9_1,Sofala,"MULTIPOLYGON (((3379518.996 -2618388.182, 3378...",210939,MOZ,Mozambique,Sofala,9703.379027,419
7,SEN.10_1,Saint-Louis,"POLYGON ((-1588223.527 1995300.864, -1588860.8...",290334,SEN,Senegal,Saint-Louis,8997.983151,29


In [21]:
gadm_level1_top[gadm_level1_top['NAME_0'].isin(top_countries.values)].to_file("../coastchange_hotspots_obselete/hotspots_gadm_level1.geojson", driver="GeoJSON")